In [60]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import branca.colormap as cm
import seaborn as sns

In [2]:
!pip3 install --upgrade gradio

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [23]:
!pip3 install geopy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [61]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for Vulcan Fossil Fuel CO₂ Emissions, Version 4. 
collection_ffco2 = "odiac-ffco2-monthgrid-v2023"
# Name of the collection for MiCASA Land Carbon Flux
collection_landcarbon = "micasa-carbonflux-daygrid-v1"
# Name of the collection for methane emission plumes 
collection_ch4plume = "emit-ch4plume-v1"

In [62]:
collection_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2}").json()
collection_landcarbon = requests.get(f"{STAC_API_URL}/collections/{collection_landcarbon}").json()
collection_ch4plume = requests.get(f"{STAC_API_URL}/collections/{collection_ch4plume}").json()

In [63]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [64]:
num_items_ffco2 = get_item_count(collection_ffco2["id"])

items_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2['id']}/items?limit={num_items_ffco2}").json()["features"]

In [65]:
# num_items_landcarbon = get_item_count(collection_landcarbon["id"])
num_items_landcarbon = 800

items_landcarbon = requests.get(f"{STAC_API_URL}/collections/{collection_landcarbon['id']}/items?limit={num_items_landcarbon}").json()["features"]

In [66]:
num_items_ch4plume = 1493

items_ch4plume = requests.get(f"{STAC_API_URL}/collections/{collection_ch4plume['id']}/items?limit={num_items_ch4plume}").json()["features"]

In [67]:
items_ffco2 = {item["properties"]["start_datetime"][:7]: item for item in items_ffco2}
items_landcarbon = {item["properties"]["datetime"][:10]: item for item in items_landcarbon}
items_ch4plume = {item["id"]: item for item in items_ch4plume}

asset_ffco2 = "co2-emissions"
asset_landcarbon = "rh"
assert_ch4plume = "ch4-plume-emissions"

In [68]:
rescale_ffco2 = {"max":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["max"], "min":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["min"]}
rescale_landcarbon = {"max":items_landcarbon[list(items_landcarbon.keys())[0]]["assets"][asset_landcarbon]["raster:bands"][0]["histogram"]["max"], "min":items_landcarbon[list(items_landcarbon.keys())[0]]["assets"][asset_landcarbon]["raster:bands"][0]["histogram"]["min"]}
rescale_ch4plume = {"max":items_ch4plume[list(items_ch4plume.keys())[0]]["assets"][assert_ch4plume]["raster:bands"][0]["histogram"]["max"], "min":items_ch4plume[list(items_ch4plume.keys())[0]]["assets"][assert_ch4plume]["raster:bands"][0]["histogram"]["min"]}

In [69]:
from geopy.geocoders import Nominatim

def search_location(location):
    geolocator = Nominatim(user_agent="user-id")
    location = geolocator.geocode(location)
    if location:
        return location.latitude, location.longitude
    else:
        # 都市名が見つからなかった場合は、デフォルトの場所（東京）を返す
        # print(f"Error: '{city}' not found. Using default location (Tokyo).")
        return 35.682839, 139.759455  # 東京の緯度・経度

In [88]:
import random
from datetime import datetime, timedelta
from branca.colormap import LinearColormap

color_map = "spectral_r"

def get_valid_tile_url(date, item_dict, asset, rescale, color_map):
    """
    指定された日付でデータが存在するか確認し、なければ次の日に移行してデータを探す。
    """
    while date <= datetime.now():  # 現在の日付までループ
        if asset == "rh":
            formatted_date = date.strftime("%Y-%m-%d")
        else:
            formatted_date = date.strftime("%Y-%m")
        try:
            tile_url = requests.get(
                f"{RASTER_API_URL}/collections/{item_dict[formatted_date]['collection']}/items/{item_dict[formatted_date]['id']}/tilejson.json?"
                f"&assets={asset}"
                f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
                f"&rescale={rescale['min']},{rescale['max']}", 
            ).json()
            return tile_url
        except KeyError:
            # 日付にデータが存在しない場合、次の日に進める
            date += timedelta(days=1)
    
    # データが見つからなかった場合
    print(f"Error: Data not found for {date.strftime('%Y-%m-%d')}.")
    return None

def year_map(city="tokyo", year=2022):
    year = str(year)
    
    # FFCO2タイルの取得
    ffco2_date = datetime.strptime(f"{year}-01", "%Y-%m")
    _ffco2_tile = get_valid_tile_url(ffco2_date, items_ffco2, asset_ffco2, rescale_ffco2, color_map)
    
    # landcarbonタイルの取得
    landcarbon_date = datetime.strptime(f"{year}-01-01", "%Y-%m-%d")
    _landcarbon_tile = get_valid_tile_url(landcarbon_date, items_landcarbon, asset_landcarbon, rescale_landcarbon, color_map)
    
    # 都市名に基づいて緯度経度を取得
    lat, lng = search_location(city)

    # FoliumのDualMapを作成
    map_ = folium.plugins.DualMap(location=(lat, lng), zoom_start=6)
    
    # カラーマップの作成
    colormap = LinearColormap(
        colors=['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'],
        vmin=0, vmax=1500
    )
    colormap.caption = 'ppm-m'
    
    color = colormap((rescale_ch4plume['max'] - rescale_ch4plume['min']) / 2)

    # items_ch4plumeに基づいてランダムに15個のキーを取得
    random_keys = random.sample(list(items_ch4plume.keys()), min(15, len(items_ch4plume)))
    
    for key in random_keys:
        methane_plume_tile = requests.get(
            f"{RASTER_API_URL}/collections/{items_ch4plume[key]['collection']}/items/{items_ch4plume[key]['id']}/tilejson.json?"
            f"&assets={assert_ch4plume}"
            f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
            f"&rescale={rescale_ch4plume['min']},{rescale_ch4plume['max']}", 
        ).json()
        
        # マーカーの作成とDualMapの両方に追加
        for map_instance in [map_.m1, map_.m2]:
            folium.Marker(
                location=[methane_plume_tile["center"][1], methane_plume_tile["center"][0]],
                popup=items_ch4plume[key]["id"],
                icon=folium.Icon(color=color),
            ).add_to(map_instance)
    
    # FFCO2タイルがあればレイヤーとして追加
    if _ffco2_tile:
        map_layer_ffco2 = TileLayer(
            tiles=_ffco2_tile["tiles"][0],
            attr="GHG",
            name=f'{year} Total CO2 Fossil Fuel Emissions',
            overlay=True,
            opacity=0.8,
        )
        map_layer_ffco2.add_to(map_.m1)
    
    # landcarbonタイルがあればレイヤーとして追加
    if _landcarbon_tile:
        map_layer_landcarbon = TileLayer(
            tiles=_landcarbon_tile["tiles"][0],
            attr="GHG",
            name=f'{year} RH Level',
            overlay=True,
            opacity=0.8,
        )
        map_layer_landcarbon.add_to(map_.m2)

    return map_._repr_html_()

In [81]:
year_map("tokyo", 2003)

/var/folders/gc/pl4_8nd9097bt5tjqkgdgdcw0000gn/T/ipykernel_16139/516941550.py:93: UserWarning: color argument of Icon should be one of: {'purple', 'black', 'beige', 'lightblue', 'darkblue', 'orange', 'gray', 'lightred', 'cadetblue', 'blue', 'pink', 'lightgray', 'darkpurple', 'darkred', 'lightgreen', 'white', 'darkgreen', 'red', 'green'}.
  icon=folium.Icon(color=color),
/var/folders/gc/pl4_8nd9097bt5tjqkgdgdcw0000gn/T/ipykernel_16139/2575265797.py:74: UserWarning: color argument of Icon should be one of: {'purple', 'black', 'beige', 'lightblue', 'darkblue', 'orange', 'gray', 'lightred', 'cadetblue', 'blue', 'pink', 'lightgray', 'darkpurple', 'darkred', 'lightgreen', 'white', 'darkgreen', 'red', 'green'}.
  icon=folium.Icon(color=color),


'<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><span style="color:#565656">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;html&gt;\n&lt;head&gt;\n    \n    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n    \n        &lt;script&gt;\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        &lt;/script&gt;\n    \n    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://code.jquery.com/jquery-3.7.1.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js&quot;&gt;&lt;/sc

In [89]:
import gradio as gr

# 画像のパス
image_path = "./../images/nasa_space_apps.png"

# カスタムCSS
custom_css = """
body {
    font-family: Inter, Public Sans, sans-serif;
    font-size: 20px !important;
}

# .header-container {
#     display: flex;
#     align-items: center;
#     margin-bottom: 20px;  /* ヘッダーと説明文の間にスペースを追加 */
# }

.nasa_logo img{
    margin-left: 50px;
    background-color: black;
}

.logo img{
    margin-right: 50px;
}

.header-container h1 {
    font-size: 40px !important;  /* タイトルのフォントサイズを指定 */
    text-align: left;           /* タイトルを中央揃え */
    margin-top: 50px;          /* タイトルの上にスペースを追加 */
}

.markdown h1 {
    font-size: 400px !important;
    margin-top: 10px;             /* デフォルトのマージンを削除 */
    margin-top: 100px;          /* デフォルトのマージンを削除 */
}

.markdown p {
    font-size: 50px !important;  /* 段落のフォントサイズを指定 */
}
"""

# Gradioインターフェースの作成
with gr.Blocks(theme=gr.themes.Default(primary_hue="cyan", secondary_hue="yellow"), css=custom_css) as demo:
    
    # ヘッダーに画像とタイトルを挿入
    with gr.Row():
        # gr.Imageコンポーネントを使用して画像を表示
        gr.Image('./../images/nasa_space_apps.png', elem_classes="nasa_logo", width="150px", height="150px", container=False, show_download_button=False, show_fullscreen_button=False)
        gr.Image('./../images/logo.png', elem_classes="logo", width="150px", height="150px", container=False, show_download_button=False, show_fullscreen_button=False)
        # gr.Markdown("<h1>CO2 Fossil Fuel Emissions Map</h1>", elem_classes="header-container")
    
    # ヘッダーの下に説明文
    gr.Markdown("""
    <h2><p>This map shows the human-caused and natural greenhouse gas emissions data over the years. Use the scroll bar to set the year and enter the location you wish to look up in the box below.</p></h2>
    """)
    
    # テキストボックスで都市名を検索
    city_input = gr.Textbox(label="Enter a city to search for GHG emission data:", placeholder="Type city name...", lines=1)
    
    # マップ表示エリア
    output = gr.HTML()
    
    # スライダー（検索バー）を配置
    year_slider = gr.Slider(2001, 2022, step=1, label="Select Year", interactive=True)
    
    city_input.submit(year_map, inputs=[city_input, year_slider], outputs=output)
    # スライダーを動かすとマップを更新
    year_slider.change(year_map, inputs=[city_input, year_slider], outputs=output)

# アプリケーションを起動
demo.launch()

Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.


/var/folders/gc/pl4_8nd9097bt5tjqkgdgdcw0000gn/T/ipykernel_16139/2575265797.py:74: UserWarning: color argument of Icon should be one of: {'purple', 'black', 'beige', 'lightblue', 'darkblue', 'orange', 'gray', 'lightred', 'cadetblue', 'blue', 'pink', 'lightgray', 'darkpurple', 'darkred', 'lightgreen', 'white', 'darkgreen', 'red', 'green'}.
  icon=folium.Icon(color=color),
